<a href="https://colab.research.google.com/github/AndreaHernandez5072/CienciaDatos/blob/main/HousePriceEstimator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report,confusion_matrix, f1_score,r2_score,mean_squared_error,mean_absolute_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier

# Cargando la data


In [55]:
df = pd.read_csv("train.csv")
dfe=pd.read_csv("Evaluation.csv")

# Identificando valores nulos


In [56]:
def display_missing_value(df):
    for col in df.columns.tolist():
        print('{} column missing values: {}'.format(col, df[col].isnull().sum()),'. (',(df[col].isnull().sum()/df.shape[0])*100,'%)')
    print('\n')

In [57]:
display_missing_value(df)

id column missing values: 0 . ( 0.0 %)
price column missing values: 0 . ( 0.0 %)
bedrooms column missing values: 0 . ( 0.0 %)
bathrooms column missing values: 0 . ( 0.0 %)
sqft_living column missing values: 0 . ( 0.0 %)
sqft_lot column missing values: 0 . ( 0.0 %)
floors column missing values: 0 . ( 0.0 %)
waterfront column missing values: 0 . ( 0.0 %)
view column missing values: 0 . ( 0.0 %)
condition column missing values: 0 . ( 0.0 %)
grade column missing values: 0 . ( 0.0 %)
sqft_above column missing values: 0 . ( 0.0 %)
sqft_basement column missing values: 0 . ( 0.0 %)
yr_built column missing values: 0 . ( 0.0 %)
yr_renovated column missing values: 0 . ( 0.0 %)
zipcode column missing values: 0 . ( 0.0 %)
lat column missing values: 0 . ( 0.0 %)
long column missing values: 0 . ( 0.0 %)
sqft_living15 column missing values: 0 . ( 0.0 %)
sqft_lot15 column missing values: 0 . ( 0.0 %)




In [58]:
display_missing_value(dfe)

id column missing values: 0 . ( 0.0 %)
bedrooms column missing values: 0 . ( 0.0 %)
bathrooms column missing values: 0 . ( 0.0 %)
sqft_living column missing values: 0 . ( 0.0 %)
sqft_lot column missing values: 0 . ( 0.0 %)
floors column missing values: 0 . ( 0.0 %)
waterfront column missing values: 0 . ( 0.0 %)
view column missing values: 0 . ( 0.0 %)
condition column missing values: 0 . ( 0.0 %)
grade column missing values: 0 . ( 0.0 %)
sqft_above column missing values: 0 . ( 0.0 %)
sqft_basement column missing values: 0 . ( 0.0 %)
yr_built column missing values: 0 . ( 0.0 %)
yr_renovated column missing values: 0 . ( 0.0 %)
zipcode column missing values: 0 . ( 0.0 %)
lat column missing values: 0 . ( 0.0 %)
long column missing values: 0 . ( 0.0 %)
sqft_living15 column missing values: 0 . ( 0.0 %)
sqft_lot15 column missing values: 0 . ( 0.0 %)




En este caso, no hay campos nulos

# Explorando la data

In [59]:
df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,18371.000000,1.837100e+04,18371.000000,18371.000000,18371.000000,1.837100e+04,18371.000000,18371.000000,18371.000000,18371.000000,18371.000000,18371.000000,18371.000000,18371.000000,18371.000000,18371.000000,18371.000000,18371.000000
mean,10834.771433,5.399958e+05,3.368243,2.113385,2079.628055,1.510525e+04,1.494666,0.007294,7.659844,1786.855207,292.772849,1970.971586,85.514398,98078.080344,47.560525,-122.214107,1985.357629,12712.953024
std,6237.186244,3.646536e+05,0.934019,0.769465,917.994443,4.215087e+04,0.540669,0.085096,1.177113,828.180612,442.699016,29.383190,404.236968,53.407807,0.138310,0.140878,685.955695,27257.274694
min,0.000000,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,460.000000,651.000000
25%,5434.000000,3.223000e+05,3.000000,1.500000,1420.000000,5.037500e+03,1.000000,0.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.472400,-122.329000,1480.000000,5100.000000
50%,10875.000000,4.500000e+05,3.000000,2.250000,1910.000000,7.614000e+03,1.500000,0.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.572000,-122.231000,1830.000000,7620.000000
75%,16222.500000,6.450000e+05,4.000000,2.500000,2550.000000,1.065350e+04,2.000000,0.000000,8.000000,2210.000000,570.000000,1997.000000,0.000000,98118.000000,47.678300,-122.124000,2360.000000,10076.000000
max,21612.000000,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


In [60]:
dfe.describe()

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,3242.000000,3242.000000,3242.000000,3242.000000,3242.000000,3242.000000,3242.000000,3242.000000,3242.000000,3242.000000,3242.000000,3242.000000,3242.000000,3242.000000,3242.000000,3242.000000,3242.000000
mean,10642.964837,3.385564,2.122532,2081.439235,15116.697717,1.492289,0.008945,7.640037,1797.091610,284.347625,1971.195250,78.100247,98077.143430,47.557378,-122.212702,1993.323257,13082.964837
std,6249.592818,0.907313,0.774181,921.107089,37016.630589,0.536196,0.094169,1.166080,827.656744,441.871798,29.321734,386.866537,54.054027,0.139985,0.140560,682.250199,27570.589561
min,2.000000,0.000000,0.500000,390.000000,609.000000,1.000000,0.000000,4.000000,390.000000,0.000000,1900.000000,0.000000,98001.000000,47.176400,-122.504000,399.000000,809.000000
25%,5298.250000,3.000000,1.750000,1430.000000,5100.000000,1.000000,0.000000,7.000000,1200.000000,0.000000,1952.000000,0.000000,98031.000000,47.461900,-122.324000,1494.000000,5120.000000
50%,10543.000000,3.000000,2.250000,1920.000000,7645.500000,1.500000,0.000000,7.000000,1580.000000,0.000000,1976.000000,0.000000,98065.000000,47.570200,-122.228000,1860.000000,7635.000000
75%,16098.250000,4.000000,2.500000,2520.000000,10800.000000,2.000000,0.000000,8.000000,2230.000000,540.000000,1997.000000,0.000000,98117.000000,47.676700,-122.127250,2370.000000,10125.000000
max,21605.000000,10.000000,6.500000,10040.000000,623779.000000,3.500000,1.000000,13.000000,7680.000000,2600.000000,2015.000000,2015.000000,98199.000000,47.777100,-121.316000,5790.000000,422967.000000


In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18371 entries, 0 to 18370
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             18371 non-null  int64  
 1   price          18371 non-null  float64
 2   bedrooms       18371 non-null  int64  
 3   bathrooms      18371 non-null  float64
 4   sqft_living    18371 non-null  int64  
 5   sqft_lot       18371 non-null  int64  
 6   floors         18371 non-null  float64
 7   waterfront     18371 non-null  int64  
 8   view           18371 non-null  object 
 9   condition      18371 non-null  object 
 10  grade          18371 non-null  int64  
 11  sqft_above     18371 non-null  int64  
 12  sqft_basement  18371 non-null  int64  
 13  yr_built       18371 non-null  int64  
 14  yr_renovated   18371 non-null  int64  
 15  zipcode        18371 non-null  int64  
 16  lat            18371 non-null  float64
 17  long           18371 non-null  float64
 18  sqft_l

Los datos a tratar son view y condition.

**View** tiene categorias bad (1), regular(2), good(3) y great(4).

**condition** tiene categoria bad(1), regular(2), acceptable(3), good(4), excelent(5)



In [62]:
dfe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3242 entries, 0 to 3241
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             3242 non-null   int64  
 1   bedrooms       3242 non-null   int64  
 2   bathrooms      3242 non-null   float64
 3   sqft_living    3242 non-null   int64  
 4   sqft_lot       3242 non-null   int64  
 5   floors         3242 non-null   float64
 6   waterfront     3242 non-null   int64  
 7   view           3242 non-null   object 
 8   condition      3242 non-null   object 
 9   grade          3242 non-null   int64  
 10  sqft_above     3242 non-null   int64  
 11  sqft_basement  3242 non-null   int64  
 12  yr_built       3242 non-null   int64  
 13  yr_renovated   3242 non-null   int64  
 14  zipcode        3242 non-null   int64  
 15  lat            3242 non-null   float64
 16  long           3242 non-null   float64
 17  sqft_living15  3242 non-null   int64  
 18  sqft_lot

# Encoding variables categoricas

In [63]:
print("-"*40)
print("Train:\n")
print(df["view"].unique())
print("-"*40)
print("Evaluation:\n")
print(dfe["view"].unique())

----------------------------------------
Train:

['bad' 'fantastic' 'good' 'great' 'regular']
----------------------------------------
Evaluation:

['bad' 'great' 'regular' 'good' 'fantastic']


In [64]:
print("-"*40)
print("Train:\n")
print(df["condition"].unique())
print("-"*40)
print("Evaluation:\n")
print(dfe["condition"].unique())

----------------------------------------
Train:

['acceptable' 'good' 'excellent' 'regular' 'bad']
----------------------------------------
Evaluation:

['acceptable' 'excellent' 'good' 'regular' 'bad']


In [65]:
reemplazoCategorias={
    "view": {"bad":1,"regular":2,"good":3,"great":4,"fantastic":5},
    "condition": {"bad":1,"regular":2,"acceptable":3,"good":4,"excellent":5}
}

df = df.replace(reemplazoCategorias)
dfe = dfe.replace(reemplazoCategorias)

In [66]:
print("-"*80)
print("Train:\n")
print(df.info())
print("-"*80)
print("Evaluation:\n")
print(dfe.info())

--------------------------------------------------------------------------------
Train:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18371 entries, 0 to 18370
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             18371 non-null  int64  
 1   price          18371 non-null  float64
 2   bedrooms       18371 non-null  int64  
 3   bathrooms      18371 non-null  float64
 4   sqft_living    18371 non-null  int64  
 5   sqft_lot       18371 non-null  int64  
 6   floors         18371 non-null  float64
 7   waterfront     18371 non-null  int64  
 8   view           18371 non-null  int64  
 9   condition      18371 non-null  int64  
 10  grade          18371 non-null  int64  
 11  sqft_above     18371 non-null  int64  
 12  sqft_basement  18371 non-null  int64  
 13  yr_built       18371 non-null  int64  
 14  yr_renovated   18371 non-null  int64  
 15  zipcode        18371 non-null  int64  
 16  lat  

In [67]:
df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,18371.000000,1.837100e+04,18371.000000,18371.000000,18371.000000,1.837100e+04,18371.000000,18371.000000,18371.000000,18371.000000,18371.000000,18371.000000,18371.000000,18371.000000,18371.000000,18371.000000,18371.000000,18371.000000,18371.000000,18371.000000
mean,10834.771433,5.399958e+05,3.368243,2.113385,2079.628055,1.510525e+04,1.494666,0.007294,1.233520,3.411409,7.659844,1786.855207,292.772849,1970.971586,85.514398,98078.080344,47.560525,-122.214107,1985.357629,12712.953024
std,6237.186244,3.646536e+05,0.934019,0.769465,917.994443,4.215087e+04,0.540669,0.085096,0.765271,0.653810,1.177113,828.180612,442.699016,29.383190,404.236968,53.407807,0.138310,0.140878,685.955695,27257.274694
min,0.000000,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,1.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,460.000000,651.000000
25%,5434.000000,3.223000e+05,3.000000,1.500000,1420.000000,5.037500e+03,1.000000,0.000000,1.000000,3.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.472400,-122.329000,1480.000000,5100.000000
50%,10875.000000,4.500000e+05,3.000000,2.250000,1910.000000,7.614000e+03,1.500000,0.000000,1.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.572000,-122.231000,1830.000000,7620.000000
75%,16222.500000,6.450000e+05,4.000000,2.500000,2550.000000,1.065350e+04,2.000000,0.000000,1.000000,4.000000,8.000000,2210.000000,570.000000,1997.000000,0.000000,98118.000000,47.678300,-122.124000,2360.000000,10076.000000
max,21612.000000,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,5.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


# Outliers

In [68]:
def CuantificaOutliers (dataset):
  for col in dataset.columns:
    q1, q3 = np.percentile(dataset[col], [25, 75])
    iqr = q3 - q1
    lower_bound = q1 - (1.5 * iqr)
    upper_bound = q3 + (1.5 * iqr)
    outlier = dataset[(dataset[col] < lower_bound)
                  | (dataset[col] > upper_bound)]
    print(col,":  (",outlier.shape[0]," / ",dataset.shape[0]," )  ",(outlier.shape[0] /  dataset.shape[0])*100,"%")

In [69]:
CuantificaOutliers(df)

id :  ( 0  /  18371  )   0.0 %
price :  ( 966  /  18371  )   5.258287518371346 %
bedrooms :  ( 466  /  18371  )   2.5366066082412497 %
bathrooms :  ( 222  /  18371  )   1.2084263240977628 %
sqft_living :  ( 482  /  18371  )   2.623700397365413 %
sqft_lot :  ( 2063  /  18371  )   11.229655435196777 %
floors :  ( 0  /  18371  )   0.0 %
waterfront :  ( 134  /  18371  )   0.7294104839148658 %
view :  ( 1801  /  18371  )   9.803494638288607 %
condition :  ( 29  /  18371  )   0.15785749278754557 %
grade :  ( 1639  /  18371  )   8.921670023406456 %
sqft_above :  ( 511  /  18371  )   2.7815578901529583 %
sqft_basement :  ( 397  /  18371  )   2.1610146426432966 %
yr_built :  ( 0  /  18371  )   0.0 %
yr_renovated :  ( 787  /  18371  )   4.283925752544772 %
zipcode :  ( 0  /  18371  )   0.0 %
lat :  ( 3  /  18371  )   0.016330085460780578 %
long :  ( 214  /  18371  )   1.1648794295356812 %
sqft_living15 :  ( 447  /  18371  )   2.4331827336563063 %
sqft_lot15 :  ( 1869  /  18371  )   10.1736432420

In [70]:
CuantificaOutliers(dfe)

id :  ( 0  /  3242  )   0.0 %
bedrooms :  ( 80  /  3242  )   2.4676125848241828 %
bathrooms :  ( 91  /  3242  )   2.8069093152375078 %
sqft_living :  ( 91  /  3242  )   2.8069093152375078 %
sqft_lot :  ( 368  /  3242  )   11.35101789019124 %
floors :  ( 0  /  3242  )   0.0 %
waterfront :  ( 29  /  3242  )   0.8945095619987662 %
view :  ( 323  /  3242  )   9.962985811227638 %
condition :  ( 1  /  3242  )   0.030845157310302282 %
grade :  ( 272  /  3242  )   8.389882788402222 %
sqft_above :  ( 99  /  3242  )   3.053670573719926 %
sqft_basement :  ( 91  /  3242  )   2.8069093152375078 %
yr_built :  ( 0  /  3242  )   0.0 %
yr_renovated :  ( 127  /  3242  )   3.9173349784083897 %
zipcode :  ( 0  /  3242  )   0.0 %
lat :  ( 0  /  3242  )   0.0 %
long :  ( 41  /  3242  )   1.2646514497223937 %
sqft_living15 :  ( 74  /  3242  )   2.282541640962369 %
sqft_lot15 :  ( 325  /  3242  )   10.024676125848242 %


Tratando los outliers

In [76]:
def ModificaOutliers (dataset,cols):
  for col in cols:
    q1, q3 = np.percentile(dataset[col], [25, 75])
    iqr = q3 - q1
    lower_bound = q1 - (1.5 * iqr)
    upper_bound = q3 + (1.5 * iqr)
    dataset[col]=np.where(dataset[col]>upper_bound,upper_bound,np.where(dataset[col]<lower_bound,lower_bound,dataset[col]))
  return dataset

In [77]:
df = ModificaOutliers(df,(["price","bedrooms","bathrooms","sqft_living","sqft_lot","waterfront","view","condition","grade","sqft_above","sqft_basement","yr_renovated","lat","long","sqft_living15","sqft_lot15"]))

In [78]:
df = ModificaOutliers(df,(["bedrooms","bathrooms","sqft_living","sqft_lot","floors","waterfront","view","condition","grade","sqft_above","sqft_basement","yr_renovated","long","sqft_living15","sqft_lot15"]))

In [79]:
CuantificaOutliers(df)

id :  ( 0  /  18371  )   0.0 %
price :  ( 0  /  18371  )   0.0 %
bedrooms :  ( 0  /  18371  )   0.0 %
bathrooms :  ( 0  /  18371  )   0.0 %
sqft_living :  ( 0  /  18371  )   0.0 %
sqft_lot :  ( 0  /  18371  )   0.0 %
floors :  ( 0  /  18371  )   0.0 %
waterfront :  ( 0  /  18371  )   0.0 %
view :  ( 0  /  18371  )   0.0 %
condition :  ( 0  /  18371  )   0.0 %
grade :  ( 0  /  18371  )   0.0 %
sqft_above :  ( 0  /  18371  )   0.0 %
sqft_basement :  ( 0  /  18371  )   0.0 %
yr_built :  ( 0  /  18371  )   0.0 %
yr_renovated :  ( 0  /  18371  )   0.0 %
zipcode :  ( 0  /  18371  )   0.0 %
lat :  ( 0  /  18371  )   0.0 %
long :  ( 0  /  18371  )   0.0 %
sqft_living15 :  ( 0  /  18371  )   0.0 %
sqft_lot15 :  ( 0  /  18371  )   0.0 %


In [80]:
CuantificaOutliers(dfe)

id :  ( 0  /  3242  )   0.0 %
bedrooms :  ( 80  /  3242  )   2.4676125848241828 %
bathrooms :  ( 91  /  3242  )   2.8069093152375078 %
sqft_living :  ( 91  /  3242  )   2.8069093152375078 %
sqft_lot :  ( 368  /  3242  )   11.35101789019124 %
floors :  ( 0  /  3242  )   0.0 %
waterfront :  ( 29  /  3242  )   0.8945095619987662 %
view :  ( 323  /  3242  )   9.962985811227638 %
condition :  ( 1  /  3242  )   0.030845157310302282 %
grade :  ( 272  /  3242  )   8.389882788402222 %
sqft_above :  ( 99  /  3242  )   3.053670573719926 %
sqft_basement :  ( 91  /  3242  )   2.8069093152375078 %
yr_built :  ( 0  /  3242  )   0.0 %
yr_renovated :  ( 127  /  3242  )   3.9173349784083897 %
zipcode :  ( 0  /  3242  )   0.0 %
lat :  ( 0  /  3242  )   0.0 %
long :  ( 41  /  3242  )   1.2646514497223937 %
sqft_living15 :  ( 74  /  3242  )   2.282541640962369 %
sqft_lot15 :  ( 325  /  3242  )   10.024676125848242 %
